# Implementazione di closure

In [ ]:
from copy import deepcopy
from functools import wraps

# assumiamo che f abbia solo argomenti posizionali, di cui il primo sia 
# quello su cui opera e che ad un certo punto sarà tale per cui f(x, …) = x

def my_closure(f):
    @wraps(f) # a cosa serve wraps? controllate la documentazione…
    def _closure(*args):
        s, *other = args
        while True:
            n = f(deepcopy(s), *other) # perché viene fatta una deep-copy del primo argomento?
            if n == s: return s
            s = n
    return _closure

In [ ]:
@my_closure
def add_smaller(S):
    return S | {x - 1 for x in S if x > 0}

In [ ]:
add_smaller({3, 5})

{0, 1, 2, 3, 4, 5}

# Implementazione di clean

In [ ]:
# Dall'handout della lezione…

from liblet import Grammar 

@my_closure
def productive(prod, G):
    for P in G.P:
        if set(P.rhs) <= prod: prod |= {P.lhs}
    return prod

@my_closure
def reachable(reach, G):
    for P in G.P:
        if P.lhs in reach: reach |= set(P.rhs)
    return reach

In [ ]:
def clean(G, good):
    return Grammar(
        G.N & good, 
        G.T & good, 
        (P for P in G.P if ({P.lhs} | set(P.rhs)) <= good), 
        G.S
    )

In [ ]:
G_small = Grammar.from_string("""
S -> A | a
A -> A B
B -> a
C -> a
B -> x
""")

In [ ]:
clean(G_small, {'A', 'B', 'S', 'a'})

Grammar(N={A, B, S}, T={a}, P=(S -> A, S -> a, A -> A B, B -> a), S=S)

# L'ordine è importante!

In [ ]:
# fig 2.27, pag. 49

G = Grammar.from_string("""
S -> A B | D E
A -> a
B -> b C
C -> c
D -> d F 
E -> e 
F -> f D
""")
G

Grammar(N={A, B, C, D, E, F, S}, T={a, b, c, d, e, f}, P=(S -> A B, S -> D E, A -> a, B -> b C, C -> c, D -> d F, E -> e, F -> f D), S=S)

In [ ]:
G0 = clean(G, productive(G.T, G))
G1 = clean(G0, reachable({G0.S}, G0))

G2 = clean(G, reachable({G.S}, G))
G3 = clean(G2, productive(G2.T, G2))

In [ ]:
G1, G3

(Grammar(N={A, B, C, S}, T={a, b, c}, P=(S -> A B, A -> a, B -> b C, C -> c), S=S),
 Grammar(N={A, B, C, E, S}, T={a, b, c, d, e, f}, P=(S -> A B, A -> a, B -> b C, C -> c, E -> e), S=S))

# Grammatiche per ww^R e ww

La prima è davvero ovvia: $S\to aa | bb | aSa | bSb$, la seconda la trovate 
in un mio [gist](https://nbviewer.jupyter.org/gist/mapio/c5fe89f55cb6499a743f912b5e53198b) dell'anno scorso.
        